In [60]:
import pandas as pd
import numpy as np
%matplotlib inline

%run utils.ipynb
df = testimon_fraud()

def prepare_dataset_nn(df):
    type_dummies = 'Type_' + df['type'].astype(str)
    df = pd.concat([df, pd.get_dummies(type_dummies)], axis=1, sort=False)
    df.drop('type', axis=1, inplace=True)
    cols = ['amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest','isFraud','isFlaggedFraud','Type_CASH_OUT','Type_TRANSFER'] #, isNight'']
    return df[cols]

df = prepare_dataset_nn(df)
df.info()

KeyError: "['newBalanceDest', 'oldBalanceDest', 'newBalanceOrig', 'oldBalanceOrig'] not in index"

**training dataset** - a set of examples used to fit the parametres
**validation dataset** - provides an unbiased evaluation of a model fit while tuning the models' hyperparameters.  This dataset can be used for regularization by early stopping 
**test dataset** is a dataset used to provide an evaluation of a final model fit on the training dataset.

In [80]:
# Generate test set
y = df['isFraud'].values # npdarray
X = df.drop('isFraud',axis=1).values # npdarray

from sklearn.model_selection import train_test_split

In [81]:
# Assigning random_state makes sure that although the process is pseudo-random, 
# it will always return the same split, which makes our work more reproducible
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Generate validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)


## Oversampling
Oversampling is useful when the dataset has imbalanced.  **Synthetic Minority Over-sampling Technique (SMOTE)** is a clever way of oversampling. This method tries to create new samples while maintaining the same decision boundaries for the classes.  

For more discussion how to better handle imbalanced dataset, I suggest to have a look at the user comments in this Kaggle post, some very good discussions there:
https://www.kaggle.com/arjunjoshua/predicting-fraud-in-financial-payment-services

In [82]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
print("Before oversampling: " + str(X_train.shape[0]))
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)
print("after oversampling: " + str(X_train.shape[0]))
# Samples are removed while maintainng the same decision boundaries?

In [90]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

# Create a simple baseline model 
# - single-layer neural network using the binary cross entropy as the loss function
model = Sequential()
model.add(Dense(1, input_dim=X.shape[1])) # single-layer with 8 dependent variables
model.add(Activation('sigmoid')) # sigmoid activation function
model.compile(loss='binary_crossentropy', # loss function is binary crossentropy for binary classification
              optimizer=SGD(lr=1e-5), # Optimizer is Stochastic Gradient Descent with a low (custom) learning rate
              metrics=['acc']) # Accuracy as the metrics

In [91]:
model.fit(X_train_res,y_train_res, epochs=5, batch_size=256, validation_data=(X_val,y_val))

W0718 16:29:00.407016 140055490680640 deprecation_wrapper.py:119] From /home/alice/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 3331258 samples, validate on 185618 samples
Epoch 1/5
3331258/3331258 [==============================] - 13s 4us/step - loss: 2.4133 - acc: 0.8502 - val_loss: 0.3178 - val_acc: 0.9800
Epoch 2/5
3331258/3331258 [==============================] - 12s 4us/step - loss: 2.2006 - acc: 0.8633 - val_loss: 0.5830 - val_acc: 0.9634
Epoch 3/5
3331258/3331258 [==============================] - 12s 4us/step - loss: 2.0911 - acc: 0.8701 - val_loss: 0.7611 - val_acc: 0.9523
Epoch 4/5
3331258/3331258 [==============================] - 12s 4us/step - loss: 1.9766 - acc: 0.8771 - val_loss: 0.5081 - val_acc: 0.9681
Epoch 5/5
3331258/3331258 [==============================] - 13s 4us/step - loss: 1.7510 - acc: 0.8910 - val_loss: 0.7365 - val_acc: 0.9538


1. Note there are more samples than the original dataset due to oversampling
2. The trainng set accuracy (balanced) is lower than the validation accuracy (imbalanced).  Accuracy in this case is flawed metrics for imbalanced datasets.  